In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

!pip install category-encoders
import category_encoders as ce

import seaborn as sns
import matplotlib.pyplot as plt
!pip install catboost

In [ ]:
df = pd.read_csv("../input/income-classification/income_evaluation.csv")

In [ ]:
df.head(5)

In [ ]:
df[' income'] = df[' income'].apply(lambda x: 0 if x == ' <=50K' else 1)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
#categorical values of dataset
print("Unique values of native country: ", df[" native-country"].unique(), end='\n\n')
print("Unique values of workclass: ", df[" workclass"].unique(), end='\n\n')
print("Unique values of education: ", df[" education"].unique(), end='\n\n')
print("Unique values of marital status: ", df[" marital-status"].unique(), end='\n\n')
print("Unique values of occupation: ", df[" occupation"].unique(), end='\n\n')
print("Unique values of relationship: ", df[" relationship"].unique(), end='\n\n')
print("Unique values of race: ", df[" race"].unique(), end='\n\n')
print("Unique values of sex: ", df[" sex"].unique(), end='\n\n')

In [ ]:
df.columns

In [ ]:
target_encoder = ce.TargetEncoder()
df[' race'] = target_encoder.fit_transform(df[' race'], df[' income'])
workclass_encoder = ce.TargetEncoder()
df[' workclass'] = workclass_encoder.fit_transform(df[' workclass'], df[' income'])
native_country_encoder = ce.TargetEncoder()
df[' native-country'] = native_country_encoder.fit_transform(df[' native-country'], df[' income'])
education_encoder = ce.TargetEncoder()
df[' education'] = education_encoder.fit_transform(df[' education'], df[' income'])
marital_encoder = ce.TargetEncoder()
df[' marital-status'] = marital_encoder.fit_transform(df[ ' marital-status'], df[' income'])
occupation_encoder = ce.TargetEncoder()
df[' occupation'] = occupation_encoder.fit_transform(df[' occupation'], df[' income'])
occupation_encoder = ce.TargetEncoder()
df[' relationship'] = occupation_encoder.fit_transform(df[' relationship'], df[' income'])

In [ ]:
# binary coding sex
df[' sex'] = df[' sex'].apply(lambda x: 1 if 'Male' else 0)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(), annot=True)

In [ ]:
from sklearn.model_selection import train_test_split
droped_columns = [' income', ' native-country', ' sex']
x_cols = [i for i in df.columns if i not in droped_columns] 

X = df[x_cols]
y = df[' income']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=42, test_size=0.01, stratify=y)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
logreg_predictions = logreg.predict(X_test)

In [ ]:
print(classification_report(y_test, logreg_predictions))

In [ ]:
from catboost import CatBoostClassifier

cat_clf = CatBoostClassifier(early_stopping_rounds=20, iterations=5000, verbose=False,).fit(X_train, y_train, eval_set=(X_val, y_val))

In [ ]:
cat_pred = cat_clf.predict(X_test)
print(classification_report(y_test, cat_pred))